# Gathering Data for our Project

## Making the API request/call

In [2]:
# installing libraries
!pip install tmdbsimple --quiet

In [1]:
#importing libraries
import tmdbsimple as tmdb
import requests
import locale
import pandas as pd

In [2]:
import requests
import pandas as pd

api_key = "c5b8aa01e09c8ea8e046e9d50e876554"
url = "https://api.themoviedb.org/3/discover/movie"

params = {
    "api_key": api_key,
    "primary_release_date.gte": "2013-01-01",
    "sort_by": "revenue.desc",
    "page": 1
}

movies = []

# Make multiple requests to fetch all movies
while True:
    response = requests.get(url, params=params)
    if response.status_code == 200:
        results = response.json()["results"]
        movies.extend(results)
        # Check if there are more pages to fetch
        if response.json()["page"] < response.json()["total_pages"]:
            params["page"] = response.json()["page"] + 1
        else:
            break
    else:
        print("Failed to fetch data")
        break

df = pd.DataFrame.from_dict(movies)
df.head()

Failed to fetch data


,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg,"[12, 878, 28]",299534,en,Avengers: Endgame,After the devastating events of Avengers: Infi...,113.382,/or06FN3Dka5tukK1e9sl16pB3iy.jpg,2019-04-24,Avengers: Endgame,False,8.3,23033
1,False,/ovM06PdF3M8wvKb06i4sjW3xoww.jpg,"[878, 12, 28]",76600,en,Avatar: The Way of Water,Set more than a decade after the events of the...,3900.586,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,2022-12-14,Avatar: The Way of Water,False,7.7,7489
2,False,/8BTsTfln4jlQrLXUBquXJ0ASQy9.jpg,"[12, 28, 878, 14]",140607,en,Star Wars: The Force Awakens,Thirty years after defeating the Galactic Empi...,51.678,/wqnLdwVXoBjKibFRR5U3y0aDUhs.jpg,2015-12-15,Star Wars: The Force Awakens,False,7.3,17944
3,False,/mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg,"[12, 28, 878]",299536,en,Avengers: Infinity War,As the Avengers and their allies have continue...,234.530,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,2018-04-25,Avengers: Infinity War,False,8.3,26738
4,False,/14QbnygCuTO0vl7CAFmPf1fgZfV.jpg,"[28, 12, 878]",634649,en,Spider-Man: No Way Home,Peter Parker is unmasked and no longer able to...,347.550,/uJYYizSuA9Y3DCs0qS4qWvHfZg4.jpg,2021-12-15,Spider-Man: No Way Home,False,8.0,17230


In [12]:
print('we have received data for {} movies, having {} features'.format(df.shape[0],df.shape[1]))

we have received data for 10000 movies, having 14 features


Now, lets use the movie ids to extract more details such as revenue, budget, runtime etc....
I'll be using the movie IDs to extract these details.

In [10]:
# creating a list for movie IDs
idz = df.id.to_list()

In [7]:
len(idz)

10000

In [39]:
# set up the TMDB API endpoint and parameters
url = "https://api.themoviedb.org/3/movie/{}"
params = {
    "api_key": "c5b8aa01e09c8ea8e046e9d50e876554"
}

# fetch data for each movie ID and store in a list
movies_data = []
for movie_id in idz:
    response = requests.get(url.format(movie_id), params=params)
    if response.status_code == 200:
        movie_data = response.json()
        movie_data['movie_id'] = movie_id  # add movie_id to movie_data
        movies_data.append(movie_data)
    else:
        print(f"Error fetching data for movie ID {movie_id}")

# create a pandas DataFrame from the list of movie data
movies_df = pd.DataFrame(movies_data, columns=['movie_id', 'budget', 'production_companies', 'production_countries', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title'])

# print the DataFrame
print(movies_df.shape)

Error fetching data for movie ID 1112123
Error fetching data for movie ID 1114185
(9998, 10)


In [40]:
movies_df.head(3)

,movie_id,budget,production_companies,production_countries,revenue,runtime,spoken_languages,status,tagline,title
0,502356,100000000,"[{'id': 33, 'logo_path': '/8lvHyhjr8oUKOOy2dKX...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",891438061,92,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,The Super Mario Bros. Movie
1,640146,200000000,"[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ...","[{'iso_3166_1': 'US', 'name': 'United States o...",473237851,125,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Witness the beginning of a new dynasty.,Ant-Man and the Wasp: Quantumania
2,594767,125000000,"[{'id': 12, 'logo_path': '/5ThIuO93vsk47oexKTS...","[{'iso_3166_1': 'US', 'name': 'United States o...",132128334,130,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Oh. My. Gods.,Shazam! Fury of the Gods


Notice, we were unable to extract details for movie IDs `1112123` and `1114185`
So, we'll drop those rows in the main `df` dataframe and merge this new information we have to it.

In [41]:
# dropping rows

df = df.drop(df[df["id"].isin([112123, 114185])].index)

To merge both dataframes on an common column,that common column on the 2 different tables has to have a common name, and the best column to use as an intersection when merging 2 dataframes is usually the id  column, reason is because each movie id is unique i.e there cannot be case when 2 movies share one id.

First, lets change the `movies_id` column in our `movies_df` to just `id`, so that we can merge.

In [48]:
movies_df = movies_df.rename(columns={'movie_id': 'id'})

Now, let's merge the dataframe

In [49]:
final_df = pd.merge(df, movies_df, on='id')

# Print the merged DataFrame
print(final_df.shape)

(9998, 23)


In [50]:
final_df.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,...,vote_count,budget,production_companies,production_countries,revenue,runtime,spoken_languages,status,tagline,title_y
0,False,/lWqjXgut48IK5f5IRbDBAoO2Epp.jpg,"[16, 12, 10751, 14, 35]",502356,en,The Super Mario Bros. Movie,"While working underground to fix a water main,...",10478.582,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,2023-04-05,...,1344,100000000,"[{'id': 33, 'logo_path': '/8lvHyhjr8oUKOOy2dKX...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",891438061,92,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,The Super Mario Bros. Movie
1,False,/8YFL5QQVPy3AgrEQxNYVSgiPEbe.jpg,"[28, 12, 878]",640146,en,Ant-Man and the Wasp: Quantumania,Super-Hero partners Scott Lang and Hope van Dy...,12579.923,/ngl2FKBlU4fhbdsrtdom9LVLBXw.jpg,2023-02-15,...,1741,200000000,"[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ...","[{'iso_3166_1': 'US', 'name': 'United States o...",473237851,125,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Witness the beginning of a new dynasty.,Ant-Man and the Wasp: Quantumania
2,False,/nDxJJyA5giRhXx96q1sWbOUjMBI.jpg,"[28, 35, 14]",594767,en,Shazam! Fury of the Gods,"Billy Batson and his foster siblings, who tran...",5900.902,/2VK4d3mqqTc7LVZLnLPeRiPaJ71.jpg,2023-03-15,...,1147,125000000,"[{'id': 12, 'logo_path': '/5ThIuO93vsk47oexKTS...","[{'iso_3166_1': 'US', 'name': 'United States o...",132128334,130,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Oh. My. Gods.,Shazam! Fury of the Gods
3,False,/ovM06PdF3M8wvKb06i4sjW3xoww.jpg,"[878, 12, 28]",76600,en,Avatar: The Way of Water,Set more than a decade after the events of the...,4708.270,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,2022-12-14,...,7458,350000000,"[{'id': 127928, 'logo_path': '/cxMxGzAgMMBhTXk...","[{'iso_3166_1': 'US', 'name': 'United States o...",2312335665,192,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Return to Pandora.,Avatar: The Way of Water
4,False,/5i6SjyDbDWqyun8klUuCxrlFbyw.jpg,"[18, 28]",677179,en,Creed III,"After dominating the boxing world, Adonis Cree...",3850.189,/cvsXj3I9Q2iyyIo95AecSd1tad7.jpg,2023-03-01,...,1161,75000000,"[{'id': 21, 'logo_path': '/5Va1Ie5c4sjfEYqixQ3...","[{'iso_3166_1': 'US', 'name': 'United States o...",269000000,116,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,You can't run from your past.,Creed III


In [56]:
final_df.shape

(9998, 23)

Lets save to csv.

In [55]:
final_df.to_csv('movies.csv', index=False)

In the next steps, I'll be proceeding to, data cleaning, exploratory data analysis, feature engineering and modelling